In [1]:
!nvidia-smi


Thu Nov 21 08:48:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:00:0F.0 Off |                  N/A |
| 27%   39C    P2              59W / 260W |    349MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Script to run code on the cluster

In [2]:
import torch
torch.cuda.is_available()

True

In [17]:
!python compute_prs.py --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --data_path datasets/imagenet --cache_dir "../cache/" --samples_per_class 3


^C
Traceback (most recent call last):
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/compute_prs.py", line 10, in <module>
    from utils.factory import create_model_and_transforms, get_tokenizer
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/__init__.py", line 2, in <module>
    from utils.factory import create_model, create_model_and_transforms, create_model_from_pretrained, get_tokenizer, create_loss
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py", line 9, in <module>
    import timm
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/__init__.py", line 2, in <module>
    from .layers import is_scriptable, is_exportable, set_scriptable, set_exportable
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/layers/__init__.py", line 8, in <module>
    from .classifier import create_classifier, ClassifierHead, NormMlpClassi

In [12]:
!python compute_text_projection.py  --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/"

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

In [15]:
!python compute_text_set_projection.py --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/" --data_path text_descriptions/laion.txt

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

# Test complete text set

In [18]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 160 --num_of_last_layers 4 --text_descriptions image_descriptions_general

^C
Traceback (most recent call last):
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/compute_complete_text_set.py", line 2, in <module>
    import numpy as np
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/numpy/__init__.py", line 374, in <module>
    _sanity_check()
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/numpy/__init__.py", line 364, in _sanity_check
    if not abs(x.dot(x) - float32(2.0)) < 1e-5:
               ^^^^^^^^
KeyboardInterrupt


In [6]:
import json

# Read JSON lines
with open("output_dir/imagenet_completeness_image_descriptions_general_top_40_heads_ViT-B-32.jsonl", "r") as json_file:
    for line in json_file:
        entry = json.loads(line)  # Parse each line as a JSON object
        layer = entry["layer"]
        head = entry["head"]
        texts = entry["embeddings_sort"]

        if entry["head"] == -1:
            print(entry.keys())
            print(entry["accuracy"])

        print(f"Layer: {layer}, Head: {head}")
        print("Texts:")
        for text in texts:
            print(text)

Layer: 8, Head: 0
Texts:
{'text': 'Futuristic transportation', 'strength_abs': 0.0024546775966882706, 'strength_rel': 6.204988479614258}
{'text': 'Image of a police car', 'strength_abs': 0.0020555234514176846, 'strength_rel': 5.195997714996338}
{'text': 'Image with holographic retro gaming aesthetics', 'strength_abs': 0.00118935015052557, 'strength_rel': 3.0064659118652344}
{'text': 'Whirling carousel at a fair', 'strength_abs': 0.0011397390626370907, 'strength_rel': 2.8810577392578125}
{'text': 'Picture snapped in the Australian deserts', 'strength_abs': 0.0009832150535658002, 'strength_rel': 2.4853928089141846}
{'text': 'Whirling amusement park ride', 'strength_abs': 0.0009690687293186784, 'strength_rel': 2.4496333599090576}
{'text': 'An image of Fiji', 'strength_abs': 0.0009641635697335005, 'strength_rel': 2.4372339248657227}
{'text': 'Antique architectural element', 'strength_abs': 0.0009530987590551376, 'strength_rel': 2.409264087677002}
{'text': 'Miniature diorama photography', '

In [25]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 80 --num_of_last_layers 1 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/1 [00:00<?, ?it/s]tensor(0.0027, device='cuda:0')
Epoch [100/3000], Loss: 0.017398
tensor(0.0065, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0109, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [200/3000], Loss: 0.015066
tensor(0.0058, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0092, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [300/3000], Loss: 0.013574
tensor(0.0057, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0079, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [400/3000], Loss: 0.012781
tensor(0.0071, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0057

In [32]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 40 --num_of_last_layers 1 --text_descriptions laion

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/1 [00:00<?, ?it/s]tensor(0.0024, device='cuda:0')
Epoch [100/3000], Loss: 0.018147
tensor(0.0072, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0110, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [200/3000], Loss: 0.018132
tensor(0.0064, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0118, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [300/3000], Loss: 0.012763
tensor(0.0074, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0054, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [400/3000], Loss: 0.011532
tensor(0.0063, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0052

In [34]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 170 --num_of_last_layers 4 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s]tensor(0.0014, device='cuda:0')
Epoch [100/3000], Loss: 0.011409
tensor(0.0051, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0063, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [200/3000], Loss: 0.011156
tensor(0.0050, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0061, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [300/3000], Loss: 0.002001
tensor(0.0010, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0010, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [400/3000], Loss: 0.001307
tensor(0.0003, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0010

In [10]:
!python compute_complete_text_set.py --device cpu --model ViT-B-32 --texts_per_head 81 --num_of_last_layers 1 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/1 [00:00<?, ?it/s]/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/algorithms_text_set.py:64: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/aten/src/ATen/native/TensorShape.cpp:3683.)
  text_norm = text_features[top_n] @ text_features[top_n].T
^C


In [40]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 40 --num_of_last_layers 4 --text_descriptions laion

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s]tensor(0.0028, device='cuda:0')
Epoch [100/3000], Loss: 0.004418
tensor(0.0023, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0021, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [200/3000], Loss: 0.004035
tensor(0.0022, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0019, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [300/3000], Loss: 0.004081
tensor(0.0021, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0020, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [400/3000], Loss: 0.003612
tensor(0.0018, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0019

In [41]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 40 --num_of_last_layers 4 --text_descriptions mscoco

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s]tensor(0.0026, device='cuda:0')
Epoch [100/3000], Loss: 0.004273
tensor(0.0019, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0023, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [200/3000], Loss: 0.004849
tensor(0.0022, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0027, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [300/3000], Loss: 0.004243
tensor(0.0020, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0023, device='cuda:0', grad_fn=<SqrtBackward0>)
Epoch [400/3000], Loss: 0.004069
tensor(0.0019, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0022